In [0]:
# 1. Завантаження даних

delta_input_path_santander = "/Volumes/workspace/default/santader_delta_dataset"
santander_df = spark.read.format("delta").load(delta_input_path_santander)

print("Rows:", santander_df.count())
display(santander_df.limit(5))

In [0]:
# 2. Вибір числових ознак + кореляційний аналіз

import numpy as np
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.functions import col

# Всі числові колонки, крім ID_code і target
numeric_cols = [c for c, t in santander_df.dtypes
                if t in ("double", "int", "float", "bigint") and c not in ("ID_code", "target")]

print("Кількість числових колонок:", len(numeric_cols))

# 10% вибірки для кореляцій, щоб не перевантажити памʼять
sample_pd = santander_df.select(numeric_cols).sample(False, 0.1, seed=42).toPandas()

corr = sample_pd.corr()
print("Кореляційна матриця (фрагмент):")
display(corr.iloc[:10, :10]) 

# Сильно корельовані пари |r| > 0.8
thr = 0.8
high_corr = []
for i, c1 in enumerate(corr.columns):
    for j, c2 in enumerate(corr.columns):
        if j > i and abs(corr.loc[c1, c2]) > thr:
            high_corr.append((c1, c2, corr.loc[c1, c2]))

print("Сильно корельовані пари (|r| > 0.8):")
for c1, c2, r in high_corr[:30]:
    print(f"{c1} - {c2}: r = {r:.3f}")

# Фільтраційний метод №1: видаляємо другу ознаку в кожній парі
to_drop_corr = set()
for c1, c2, r in high_corr:
    # залишаємо першу, другу вважаємо надлишковою
    to_drop_corr.add(c2)

numeric_cols_corr = [c for c in numeric_cols if c not in to_drop_corr]
print("Після видалення сильно корельованих ознак:", len(numeric_cols_corr))

santander_df = santander_df.select(
    *[c for c in santander_df.columns if c not in to_drop_corr]  # дропаємо на рівні DataFrame
)


In [0]:
# 3. Інженерія ознак (Feature Engineering, ≥5 нових ознак)

from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf, sqrt, pow

# 3.1. Статистичні ознаки по рядку: mean, std, skewness, kurtosis
fe_cols = numeric_cols_corr[:] 

def row_skew(values):
    arr = np.array(values, dtype=float)
    if len(arr) == 0 or np.std(arr) == 0:
        return 0.0
    return float(((arr - arr.mean())**3).mean() / (arr.std()**3 + 1e-9))

def row_kurt(values):
    arr = np.array(values, dtype=float)
    if len(arr) == 0 or np.std(arr) == 0:
        return 0.0
    return float(((arr - arr.mean())**4).mean() / (arr.std()**4 + 1e-9))

skew_udf = udf(row_skew, DoubleType())
kurt_udf = udf(row_kurt, DoubleType())

# Масив ознак для рядка
santander_df = santander_df.withColumn("features_array", F.array(*[col(c) for c in fe_cols]))

santander_df = (
    santander_df
    .withColumn(
        "row_mean",
        F.expr("aggregate(features_array, 0D, (acc, x) -> acc + x) / size(features_array)")
    )
    .withColumn(
        "row_std",
        F.expr(
            "sqrt(aggregate(transform(features_array, x -> pow(x - row_mean, 2)), "
            "0D, (acc, x) -> acc + x) / size(features_array))"
        )
    )
    .withColumn("row_skew", skew_udf(col("features_array")))
    .withColumn("row_kurt", kurt_udf(col("features_array")))
)

# 3.2. Interaction features (x1 * x2, x3 * x4)
interaction_pairs = list(zip(fe_cols[:6:2], fe_cols[1:6:2]))
for f1, f2 in interaction_pairs:
    new_name = f"{f1}_x_{f2}"
    santander_df = santander_df.withColumn(new_name, col(f1) * col(f2))

# 3.3. Distance-based feature (відстань у підпросторі перших 4 змінних)
if len(fe_cols) >= 4:
    santander_df = santander_df.withColumn(
        "distance_feat",
        sqrt(
            pow(col(fe_cols[0]) - col(fe_cols[1]), 2) +
            pow(col(fe_cols[2]) - col(fe_cols[3]), 2)
        )
    )

# Прибираємо допоміжний масив
santander_df = santander_df.drop("features_array")

# Оновлюємо список числових колонок (додаємо нові фічі)
new_fe_cols = ["row_mean", "row_std", "row_skew", "row_kurt", "distance_feat"] + \
              [f"{f1}_x_{f2}" for f1, f2 in interaction_pairs]

numeric_cols_fe = numeric_cols_corr + [c for c in new_fe_cols if c in santander_df.columns]

print("Кількість числових ознак після FE:", len(numeric_cols_fe))



In [0]:
# 4. Обробка викидів (IQR) – для кількох колонок
# перші 10 числових колонок
iqr_cols = numeric_cols_corr[:10]

iqr_bounds = {}
for c in iqr_cols:
    q1, q3 = santander_df.approxQuantile(c, [0.25, 0.75], 0.01)
    iqr = q3 - q1
    low, high = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    iqr_bounds[c] = (low, high)

print("IQR межі (фрагмент):")
for c, (low, high) in list(iqr_bounds.items())[:5]:
    print(f"{c}: [{low:.4f}, {high:.4f}]")

# Фільтруємо викиди тільки по цих кількох колонках
for c, (low, high) in iqr_bounds.items():
    santander_df = santander_df.filter((col(c) >= low) & (col(c) <= high))

print("Рядків після фільтрації IQR:", santander_df.count())


In [0]:
# 5. Вибір ознак (Feature Selection: Variance + L1)

from pyspark.ml.feature import VectorAssembler
import pyspark.sql.functions as F

# 5.1. Фільтраційний метод №2: Variance + top-K
var_row = santander_df.select(
    [F.variance(col(c)).alias(c) for c in numeric_cols_fe]
).collect()[0].asDict()

var_items = [(c, float(var_row.get(c) or 0.0)) for c in numeric_cols_fe]
var_items_sorted = sorted(var_items, key=lambda x: x[1], reverse=True)

TOP_K = 30 
topk_feats = [c for c, _ in var_items_sorted[:TOP_K]]

print("Топ-30 ознак за дисперсією:")
for c, v in var_items_sorted[:10]:
    print(f"{c}: var = {v:.6f}")

# 5.2. стандартизація (z-score) тільки для top-k ознак
stats = santander_df.select(
    *[F.mean(col(c)).alias(f"{c}_mean") for c in topk_feats],
    *[F.stddev(col(c)).alias(f"{c}_std") for c in topk_feats]
).collect()[0].asDict()

scaled_cols = []
for c in topk_feats:
    mean_val = stats.get(f"{c}_mean") or 0.0
    std_val = stats.get(f"{c}_std") or 1.0
    if std_val == 0:
        std_val = 1.0
    scaled_name = f"{c}_scaled"
    santander_df = santander_df.withColumn(scaled_name, (col(c) - mean_val) / std_val)
    scaled_cols.append(scaled_name)

# вектор scaled_features
assembler_scaled = VectorAssembler(inputCols=scaled_cols, outputCol="scaled_features")
santander_df = assembler_scaled.transform(santander_df)

print(f"Розмірність scaled_features: {len(scaled_cols)}")

# 5.3. метод: L1 Logistic Regression
from pyspark.ml.classification import LogisticRegression

santander_l1 = santander_df.select("target", "scaled_features").where(col("target").isNotNull())

lr = LogisticRegression(
    featuresCol="scaled_features",
    labelCol="target",
    regParam=0.1,
    elasticNetParam=1.0  # L1
)

lr_model = lr.fit(santander_l1)

coef = lr_model.coefficients.toArray()
feature_importance_l1 = sorted(
    [(f, float(w)) for f, w in zip(topk_feats, coef)],
    key=lambda x: abs(x[1]),
    reverse=True
)[:10]

print("Топ-10 важливих ознак за L1 Logistic Regression:")
for name, weight in feature_importance_l1:
    print(f"{name}: {weight:.6f}")



In [0]:
# 6. PCA

import numpy as np
import pandas as pd
from pyspark.sql.functions import col

# топ-20 фіч після Variance Selection
top20 = ['var_27', 'var_17', 'var_84', 'var_178', 'var_176',
         'var_93', 'var_38', 'var_68', 'var_85', 'var_83',
         'var_73', 'var_58', 'var_182', 'var_137', 'var_160',
         'var_64', 'var_39', 'var_192', 'var_163', 'var_104']

# Вибірка
sample_pd = santander_df.select(top20).sample(fraction=0.03, seed=42).toPandas()

print("Sample shape:", sample_pd.shape)

# Стандартизація
from sklearn.preprocessing import StandardScaler
scaler_np = StandardScaler()
X_scaled = scaler_np.fit_transform(sample_pd.values)

# PCA
from sklearn.decomposition import PCA
pca_np = PCA(n_components=5)
X_pca = pca_np.fit_transform(X_scaled)

print("\nПояснена дисперсія:", pca_np.explained_variance_ratio_)
print("Сумарна пояснена:", pca_np.explained_variance_ratio_.sum())


pca_pdf = pd.DataFrame(X_pca, columns=[f"pca_{i+1}" for i in range(5)])
pca_spark_df = spark.createDataFrame(pca_pdf)

from pyspark.sql import functions as F
santander_sample = santander_df.sample(fraction=0.03, seed=42) \
                               .withColumn("row_idx", F.monotonically_increasing_id())

pca_spark_df = pca_spark_df.withColumn("row_idx", F.monotonically_increasing_id())

santander_pca_final = santander_sample.join(pca_spark_df, on="row_idx").drop("row_idx")

display(santander_pca_final.limit(5))

pca_sample_pd = pd.DataFrame(X_pca, columns=[f"pca_{i+1}" for i in range(5)])

In [0]:
%pip install umap-learn

In [0]:
import umap

umap_model = umap.UMAP(n_components=2, random_state=42)
X_umap = umap_model.fit_transform(X_pca)

print("UMAP виконано успішно. Розмірність:", X_umap.shape)

pca_sample_pd["umap_1"] = X_umap[:, 0]
pca_sample_pd["umap_2"] = X_umap[:, 1]

display(pca_sample_pd[["umap_1", "umap_2"]].head())


In [0]:
# Перетворюємо UMAP-таблицю назад у Spark
umap_pdf = pca_sample_pd[["umap_1", "umap_2"]]
umap_spark_df = spark.createDataFrame(umap_pdf)

umap_spark_df = umap_spark_df.withColumn("row_idx", F.monotonically_increasing_id())
santander_pca_final = santander_pca_final.withColumn("row_idx", F.monotonically_increasing_id())

# PCA + UMAP
santander_final_df = santander_pca_final.join(umap_spark_df, on="row_idx").drop("row_idx")

display(santander_final_df.limit(5))


In [0]:
output_path_santander_final = "/Volumes/workspace/default/santader_delta_3"

santander_final_df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(output_path_santander_final)

print("Фінальний датасет Santander з PCA + UMAP успішно збережений!")
